In [1]:
#import
#python 3.7 version
#tensorflow 2.3 version
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

#InceptionV3
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor

#tensorflow image processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

#make callback to get valiable metrics
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import (f1_score, precision_score, recall_score)


#from collections import Counter
import itertools
import numpy as np 
import pandas as pd

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#make train set, validation set by split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

#make date to save models as filename
from datetime import datetime

#make chart to visulaize
import matplotlib.pyplot as plt

#ignore error code, it`s not recommend at editing
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_folder = 'C:\\Users\\BVer\\dog_\\YJ_Kim\\dataset\\images1111111111ori'
train_dogs = pd.read_csv('C:\\Users\\BVer\\dog_\\YJ_Kim\\dataset\\data_ori.csv')
train_dogs.head()

,path,label
0,C:\Users\BVer\dog_\YJ_Kim\dataset\images111111...,affenpinscher
1,C:\Users\BVer\dog_\YJ_Kim\dataset\images111111...,affenpinscher
2,C:\Users\BVer\dog_\YJ_Kim\dataset\images111111...,affenpinscher
3,C:\Users\BVer\dog_\YJ_Kim\dataset\images111111...,affenpinscher
4,C:\Users\BVer\dog_\YJ_Kim\dataset\images111111...,affenpinscher


In [3]:
target_labels = train_dogs['label']

In [4]:
# One hot code the labels - need this for the model
one_hot = pd.get_dummies(target_labels, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [ ]:
train_data = np.array([img_to_array(load_img(img, target_size=(200, 200))) for img in train_dogs['path'].values.tolist()]).astype('float32')

In [ ]:
x_train, x_validation, y_train, y_validation = train_test_split(train_data, target_labels, test_size=0.2, stratify=np.array(target_labels), random_state=100)

In [ ]:
print ('x_train shape = ', x_train.shape)
print ('x_validation shape = ', x_validation.shape)

In [ ]:
data = y_train.value_counts().sort_index().to_frame()   # this creates the data frame with train numbers
data.columns = ['train']   # give the column a name
data['validation'] = y_validation.value_counts().sort_index().to_frame()   # add the validation numbers
new_plot = data[['train','validation']].sort_values(['train']+['validation'], ascending=False)   # sort the data
new_plot.plot(kind='bar', stacked=True)
plt.show()

In [ ]:
y_train = pd.get_dummies(y_train.reset_index(drop=True)).values
y_validation = pd.get_dummies(y_validation.reset_index(drop=True)).values

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=30, 
                                   # zoom_range = 0.3, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')
train_generator = train_datagen.flow(x_train, y_train, shuffle=False, batch_size=32, seed=10)

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = train_datagen.flow(x_validation, y_validation, shuffle=False, batch_size=32, seed=10)

In [ ]:
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(200, 200, 3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
x = Dense(512, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

In [ ]:
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
#model.compile(Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
trial = 2
epochs = 20
metrics = Metrics(val_generator, validation_steps=10)
history = model.fit_generator(
    generator = train_generator,
    #steps_per_epoch = len(train_generator), // 10,
    validation_data = val_generator,
    #validation_steps = len(val_generator), // 10,
    epochs = epochs,
)
model.save_weights(datetime.today().strftime("%Y%m%d")+ '_weights_' +str(epochs) + '_' + str(trial) +'.h5')
model.save(datetime.today().strftime("%Y%m%d")+ '_model_'+ str(epochs) + '_' + str(trial) +'.h5')

In [ ]:
plt.plot(history.history['loss'], label = 'Loss')
plt.plot(history.history['val_loss'], label = 'Val_Loss')
plt.plot(history.history['accuracy'], label = 'Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Val_Accuracy')
plt.legend()
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

In [ ]:
scores = model.evaluate_generator(val_generator, steps=5)

In [ ]:
print("-- Evaluate --")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
base_model.trainable = True
#model.summary()

model.compile(Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)
model.save_weights(datetime.today().strftime("%Y%m%d")+ '_weights_ft_' +str(epochs) + '_' + str(trial) +'.h5')
model.save(datetime.today().strftime("%Y%m%d")+ '_model_ft_'+ str(epochs) + '_' + str(trial) +'.h5')

In [ ]:
print("-- Evaluate --")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
plt.plot(history.history['loss'], label = 'Loss')
plt.plot(history.history['val_loss'], label = 'Val_Loss')
plt.plot(history.history['accuracy'], label = 'Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Val_Accuracy')
plt.legend()
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')